## Paquets i dades originals ## 

In [ ]:
import json
# TODO: solo poner las librerias que realmente se usan
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list, Main_respiratory_infections_list, LRTI_list, COPD_exacerbations_list, Pulmonary_fibrosis_fibrotorax_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtenir_ultim_resultat, obtenir_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple
# TODO hi han dos funcions que no es diuen de la manera que les crides, revisa-ho

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [ ]:
# TODO: la asignación va aquí
taula.head()

Distribuiré les dades en 3 grups (més un grup que conté la totalitat de dades):
- **AMB_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **AMB_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Han sigut diagnosticats amb el codi ICD de pneumònia
    - Han passat menys de 30 dies entre el diagnòstic amb l'ICD de pneumònia i el resultat positiu al MECVV
- **SENSE_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han passat més de 30 dies entre el diagnòstic amb l'ICD de pneumònia i  el resultat positiu del MECVV
- **TOTAL**: aquest grup conté tots els pacients de l'estudi *#TODO: hacer descriptiva de este grupo*

# AMB_PA 

In [ ]:
AMB_PA = taula[taula["PA_diagnosticada"] == 1.0]
AMB_PA.head()
# TODO: no es mostra mai tot el dataset, mostra nomes 5 files tal i com esta predefinit en la llibreria

# AMB_PA_MECVV

In [ ]:
AMB_PA_MECVV = taula[
    (taula["dias entre primer ICD pneumonia y primer MECVV positivo"] < 30)]
AMB_PA_MECVV.head()

# SENSE_PA

In [ ]:
SENSE_PA = taula[
           (taula["dias entre primer ICD pneumonia y primer MECVV positivo"] > 30)  (taula['P_diagnosticada'] == 1.0)]
SENSE_PA.head()

In [ ]:
#TODO: si quito el | (taula['P_diagnosticada'] == 1.0) se me quedan en 8 pacientes

# TOTAL

In [ ]:
TOTAL = taula
taula.head()

Ara que tinc totes les dades classificades en aquests grups, faré una descriptiva. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat per mirar la seva distribució, la qual esperaré que sigui similar entre el grup AMB_PA i AMB_PA_MECVV, i diferent entre SENSE_PA i els altres 2 grups. Els criteris que seguiré per utilitzar un test o un altre és, primer, fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.


## Nombre de pacients

In [ ]:
num_AMB_PA = len(AMB_PA)
num_AMB_PA_MECVV= len(AMB_PA_MECVV)
num_SENSE_PA = len(SENSE_PA)
num_TOTAL = len(TOTAL)

print(num_AMB_PA, num_AMB_PA_MECVV, num_SENSE_PA)
# TODO: Genera la frase desde el print, no la generis despres fent hard code. No sabrás mai el numero real fins la ultima proba
# exemple: print(f"El número de pacients en AMB_PA és de {num_AMB_PA}, en AMB_PA_MECVV és de {num_AMB_PA_MECVV} i en SENSE_PA és de {num_SENSE_PA}")

Això també es pot comprovar en observar les files que conté les taules anteriors de cada grup, on veiem que la taula de AMB_PA té 17 files, la de AMB_PA_MECV 10 i la de SENSE_PA 49. 

#### Pvalor

##### Normalitat

In [578]:
stat, p_value = shapiro(range(num_AMB_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9621
Valor p = 0.6704
El nombre de files segueix una distribució normal.


In [579]:
stat, p_value = shapiro(range(num_AMB_PA_MECVV))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9702
Valor p = 0.8924
El nombre de files segueix una distribució normal.


In [ ]:
stat, p_value = shapiro(range(num_SENSE_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(range(num_TOTAL))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [581]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i AMB_PA_MECVV: 0.5246
No existeixen diferències significatives entre els 2 grups.


In [582]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA i SENSE_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA i SENSE_PA: 0.7664
No existeixen diferències significatives entre els 2 grups.


In [583]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV.index, SENSE_PA.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i SENSE_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.5246
No existeixen diferències significatives entre els 2 grups.


In [ ]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA.index, AMB_PA_MECVV.index, equal_var=False)
print(f"P-valor entre AMB_PA i AMB_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_14, p_val_14 = ttest_ind(AMB_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_24, p_val_24 = ttest_ind(AMB_PA_MECVV.index, TOTAL.index, equal_var=False)
print(f"P-valor entre AMB_PA_MECVV i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [ ]:
t_stat_34, p_val_34 = ttest_ind(SENSE_PA.index, TOTAL.index, equal_var=False)
print(f"P-valor entre SENSE_PA i TOTAL: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

## Edat

In [584]:
edad_AMB_PA = AMB_PA['edat'].mean()
desviacion_estandar_edad = AMB_PA['edat'].std()

resultado = f"{edad_AMB_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)


Mitjana ± Desviació Estàndard de l'edat: 87.06 ± 8.99


In [585]:
edad_AMB_PA_MECVV = AMB_PA_MECVV['edat'].mean()
desviacion_estandar_edad = AMB_PA_MECVV['edat'].std()

resultado = f"{edad_AMB_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 88.90 ± 8.67


In [ ]:
edad_SENSE_PA = SENSE_PA['edat'].mean()
desviacion_estandar_edad = SENSE_PA['edat'].std()

resultado = f"{edad_SENSE_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

In [ ]:
edad_TOTAL = TOTAL['edat'].mean()
desviacion_estandar_TOTAL = TOTAL['edat'].std()

resultado = f"{edad_SENSE_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

#### Pvalor

##### Normalitat 

In [587]:
stat, p_value = shapiro(AMB_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9410
Valor p = 0.3297
El nombre de files segueix una distribució normal.


In [588]:
stat, p_value = shapiro(AMB_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9057
Valor p = 0.2528
El nombre de files segueix una distribució normal.


In [ ]:
stat, p_value = shapiro(SENSE_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [590]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['edat'], AMB_PA_MECVV['edat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.6051223082851969


In [591]:
stat, p_value_13 = mannwhitneyu(AMB_PA['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.1815160884632735


In [592]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['edat'], SENSE_PA['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.11458529353557328


In [ ]:
stat, p_value_14 = mannwhitneyu(AMB_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

In [ ]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

In [ ]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['edat'], TOTAL['edat'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

## Sexe

In [593]:
sexe_AMB_PA = AMB_PA['sexe'].value_counts()
sexe_AMB_PA_MECVV = AMB_PA_MECVV['sexe'].value_counts()
sexe_SENSE_PA = SENSE_PA['sexe'].value_counts()
sexe_TOTAL = TOTAL['sexe'].value_counts()

In [594]:
if 'F' in sexe_AMB_PA:
    percent_female1 = (sexe_AMB_PA['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA")

Percentatge de sexe femení en AMB_PA: 47.06%


In [595]:
if 'F' in sexe_AMB_PA_MECVV:
    percent_female2 = (sexe_AMB_PA_MECVV['F'] / len(AMB_PA)) * 100
    print(f"Percentatge de sexe femení en AMB_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en AMB_PA_MECVV")

Percentatge de sexe femení en AMB_PA_MECVV: 47.06%


In [596]:
if 'F' in sexe_SENSE_PA:
    percent_female3 = (sexe_SENSE_PA['F'] / len(SENSE_PA)) * 100
    print(f"Percentatge de sexe femení en SENSE_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SENSE_PA")


Percentatge de sexe femení en SENSE_PA: 47.06%


In [ ]:
if 'F' in sexe_TOTAL:
    percent_female3 = (sexe_TOTAL['F'] / len(TOTAL)) * 100
    print(f"Percentatge de sexe femení en TOTAL: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en TOTAL")

#### Pvalor

##### Normalitat

Al ser una variable categòrica no es pot comprovar la seva normalitat. Realitzaré directament el test Xi-quadrat

In [597]:
contingency_table_AMB_PA_AMB_PA_MECVV = pd.crosstab(index=AMB_PA['sexe'], columns=AMB_PA_MECVV['sexe'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_AMB_PA_AMB_PA_MECVV)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.6666666666666665
P-valor entre AMB_PA i AMB_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.


In [598]:
contingency_table_AMB_PA_SENSE_PA = pd.crosstab(index=AMB_PA['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 13.223042052469136
P-valor entre AMB_PA i SENSE_PA: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.


In [599]:
contingency_table_AMB_PA_MECVV_SENSE_PA = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=SENSE_PA['sexe'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_AMB_PA_MECVV_SENSE_PA)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA: 6.267361111111111
P-valor entre AMB_PA_MECVV i SENSE_PA: 0.012298217061234292
Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i SENSE_PA.


In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.")

In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['sexe'], columns=TOTAL['sexe'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['sexe'], columns=TOTAL['sexe'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.")

## Ingressos (admissions)

In [600]:
promedio_ingresos = AMB_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = AMB_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.59 ± 3.37


In [601]:
promedio_ingresos = AMB_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = AMB_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.90 ± 2.08


In [ ]:
promedio_ingresos = SENSE_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SENSE_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

In [ ]:
promedio_ingresos = TOTAL['Num_ingresos'].mean()
desviacion_estandar_ingresos = TOTAL['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

#### Pvalor

##### Normalitat

In [603]:
stat, p_value = shapiro(AMB_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8008
Valor p = 0.0021
El nombre de files no segueix una distribució normal.


In [604]:
stat, p_value = shapiro(AMB_PA_MECVV['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9558
Valor p = 0.7376
El nombre de files segueix una distribució normal.


In [ ]:
stat, p_value = shapiro(SENSE_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(TOTAL['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [606]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Num_ingresos'], AMB_PA_MECVV['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.3599581940859352


In [607]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Num_ingresos'], SENSE_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.514931142191354


In [608]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Num_ingresos'], SENSE_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.7831397976578486


In [ ]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Num_ingresos'], TOTAL['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

In [ ]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Num_ingresos'], TOTAL['Num_ingresos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

In [ ]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Num_ingresos'], TOTAL['Num_ingresos'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

## Disfàgia

In [609]:
disf_AMB_PA= (AMB_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA:.2f}%")

Percentatge de disfàgia: 47.06%


In [610]:
disf_AMB_PA_MECVV = (AMB_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_AMB_PA_MECVV:.2f}%")

Percentatge de disfàgia: 30.00%


In [611]:
disf_SENSE_PA= (SENSE_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SENSE_PA:.2f}%")

Percentatge de disfàgia: 18.37%


In [ ]:
disf_TOTAL= (TOTAL['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_TOTAL:.2f}%")

#### Pvalor

##### Normalitat

In [612]:
stat, p_value = shapiro(AMB_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [613]:
stat, p_value = shapiro(AMB_PA_MECVV['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5942
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [614]:
stat, p_value = shapiro(SENSE_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4716
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [ ]:
stat, p_value = shapiro(TOTAL['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
contingency_table_12 = pd.crosstab(index=AMB_PA['DO_diagnosticada'], columns=AMB_PA_MECVV['DO_diagnosticada'])

chi2_AMB_PA_AMB_PA_MECVV, p_value_AMB_PA_AMB_PA_MECVV, dof_AMB_PA_AMB_PA_MECVV, expected_AMB_PA_AMB_PA_MECVV = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_AMB_PA_AMB_PA_MECVV)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_AMB_PA_AMB_PA_MECVV)

if p_value_AMB_PA_AMB_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i AMB_PA_MECVV.")

In [ ]:
contingency_table_13 = pd.crosstab(index=AMB_PA['DO_diagnosticada'], columns=SENSE_PA['DO_diagnosticada'])

chi2_AMB_PA_SENSE_PA, p_value_AMB_PA_SENSE_PA, dof_AMB_PA_SENSE_PA, expected_AMB_PA_SENSE_PA = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_AMB_PA_SENSE_PA)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_AMB_PA_SENSE_PA)

if p_value_AMB_PA_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i SENSE_PA.")

In [ ]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['DO_diagnosticada'], columns=SENSE_PA['DO_diagnosticada'])

chi2_AMB_PA_MECVV_SENSE_PA, p_value_AMB_PA_MECVV_SENSE_PA, dof_AMB_PA_MECVV_SENSE_PA, expected_AMB_PA_MECVV_SENSE_PA = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVVV i SENSE_PA:", chi2_AMB_PA_MECVV_SENSE_PA)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_AMB_PA_MECVV_SENSE_PA)

if p_value_AMB_PA_MECVV_SENSE_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i SENSE_PA.")

In [616]:
contingency_table_14 = pd.crosstab(index=AMB_PA['DO_diagnosticada'], columns=TOTAL['DO_diagnosticada'])

chi2_AMB_PA_TOTAL, p_value_AMB_PA_TOTAL, dof_AMB_PA_TOTAL, expected_AMB_PA_TOTAL = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_AMB_PA_TOTAL)
print("P-valor entre AMB_PA i TOTAL:", p_value_AMB_PA_TOTAL)

if p_value_AMB_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA i TOTAL.")

P-valor entre AMB_PA i SENSE_PA 0.02125060274444335


In [ ]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['DO_diagnosticada'], columns=TOTAL['DO_diagnosticada'])

chi2_AMB_PA_MECVV_TOTAL, p_value_AMB_PA_MECVV_TOTAL, dof_AMB_PA_MECVV_TOTAL, expected_AMB_PA_MECVV_TOTAL = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_AMB_PA_MECVV_TOTAL)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_AMB_PA_MECVV_TOTAL)

if p_value_AMB_PA_MECVV_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en AMB_PA_MECVV i TOTAL.")

In [ ]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['DO_diagnosticada'], columns=TOTAL['DO_diagnosticada'])

chi2_SENSE_PA_TOTAL, p_value_SENSE_PA_TOTAL, dof_SENSE_PA_TOTAL, expected_SENSE_PA_TOTAL = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_SENSE_PA_TOTAL)
print("P-valor entre SENSE_PA i TOTAL:", p_value_SENSE_PA_TOTAL)

if p_value_SENSE_PA_TOTAL < 0.05:
    print("Hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de DO en SENSE_PA i TOTAL.")

## Index de Barthel

### Promedio

In [618]:
promedio_barthel = AMB_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = AMB_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 22.65 ± 24.63


In [619]:
promedio_barthel = AMB_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = AMB_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 28.50 ± 22.24


In [620]:
promedio_barthel = SENSE_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SENSE_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 48.88 ± 37.42


#### Pvalor

##### Normalitat

In [621]:
stat, p_value = shapiro(AMB_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8517
Valor p = 0.0114
El nombre de files no segueix una distribució normal.


In [622]:
stat, p_value = shapiro(AMB_PA_MECVV['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9138
Valor p = 0.3083
El nombre de files segueix una distribució normal.


In [623]:
stat, p_value = shapiro(SENSE_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8858
Valor p = 0.0002
El nombre de files no segueix una distribució normal.


In [624]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'], AMB_PA_MECVV['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.41631085084208264


In [625]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'], SENSE_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.012373091641540783


In [626]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'], SENSE_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.13803563369050034


### Valors de dependència

##### AMB_PA

In [627]:
indepe_AMB_PA = (AMB_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA: {indepe_AMB_PA:.2f}%")

Percentatge de indepe_AMB_PA: 0.00%


In [628]:
mod_AMB_PA = (AMB_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_AMB_PA: {mod_AMB_PA:.2f}%")

Percentatge de mod_AMB_PA: 5.88%


In [629]:
sev_AMB_PA = (AMB_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_AMB_PA: {sev_AMB_PA:.2f}%")

Percentatge de sev_AMB_PA: 41.18%


In [630]:
total_AMB_PA = (AMB_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_AMB_PA: {total_AMB_PA:.2f}%")

Percentatge de total_AMB_PA: 52.94%


##### AMB_PA_MECVV

In [631]:
indepe_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_AMB_PA_MECVV: {indepe_AMB_PA_MECVV:.2f}%")

Percentatge de indepe_AMB_PA_MECVV: 0.00%


In [632]:
mod_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_AMB_PA_MECVV: {mod_AMB_PA_MECVV:.2f}%")

Percentatge de mod_AMB_PA_MECVV: 0.00%


In [633]:
sev_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_AMB_PA_MECVV: {sev_AMB_PA_MECVV:.2f}%")

Percentatge de sev_AMB_PA_MECVV: 60.00%


In [634]:
total_AMB_PA_MECVV = (AMB_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Percentatge de total_AMB_PA_MECVV: {total_AMB_PA_MECVV:.2f}%")

Percentatge de total_AMB_PA_MECVV: 40.00%


##### SENSE_PA

In [635]:
indepe_SENSE_PA = (SENSE_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SENSE_PA: {indepe_SENSE_PA:.2f}%")

Percentatge de indepe_SENSE_PA: 20.41%


In [636]:
mod_SENSE_PA = (SENSE_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SENSE_PA: {mod_SENSE_PA:.2f}%")

Percentatge de mod_SENSE_PA: 18.37%


In [637]:
sev_SENSE_PA = (SENSE_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SENSE_PA: {sev_SENSE_PA:.2f}%")

Percentatge de sev_SENSE_PA: 28.57%


In [638]:
total_SENSE_PA = (SENSE_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Percentatge de total_SENSE_PA: {total_SENSE_PA:.2f}%")

Percentatge de total_SENSE_PA: 32.65%


#### Pvalor

##### Independència

In [639]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'] == 100.0, AMB_PA_MECVV['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV:", p_value_12)

P-valor entre indepe_AMB_PA i indepe_AMB_PA_MECVV: 1.0


In [640]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'] == 100.0, SENSE_PA['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_AMB_PA i indepe_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i indepe_SENSE_PA: 0.04605091215969595


In [641]:
stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Barthel_resultados'] == 100.0, SENSE_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_AMB_PA_MECVV i SENSE_PA:", p_value_23)

P-valor entre indepe_AMB_PA_MECVV i SENSE_PA: 0.13803563369050034


##### Moderada

In [642]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(61, 95), AMB_PA_MECVV['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV:", p_value_12)

P-valor entre mod_AMB_PA i mod_AMB_PA_MECVV: 0.49002535950179893


In [643]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(61, 95), SENSE_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_AMB_PA i mod_SENSE_PA:", p_value_13)

P-valor entre indepe_AMB_PA i mod_SENSE_PA: 0.22404485220846948


In [644]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'].between(61, 95), SENSE_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA :", p_value_23)

P-valor entre mod_AMB_PA_MECVV i mod_SENSE_PA : 0.14889565499123467


##### Severa

In [645]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(21,60), AMB_PA_MECVV['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV:", p_value_12)

P-valor entre sev_AMB_PA i sev_AMB_PA_MECVV: 0.3688417252786701


In [646]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'].between(21,60), SENSE_PA['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_AMB_PA i sev_SENSE_PA:", p_value_13)

P-valor entre sev_AMB_PA i sev_SENSE_PA: 0.3446368758082651


In [647]:
t_stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'].between(21,60), SENSE_PA['Barthel_resultados'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA:", p_value_23)

P-valor entre sev_AMB_PA_MECVV i sev_SENSE_PA: 0.05945988532601827


##### Total

In [648]:
stat_12, p_value_12 = mannwhitneyu(AMB_PA['Barthel_resultados'] <= 20.0, AMB_PA_MECVV['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_AMB_PA_MECVV:", p_value_12)

P-valor entre total_AMB_PA i total_AMB_PA_MECVV: 0.5426838040005246


In [649]:
stat_13, p_value_13 = mannwhitneyu(AMB_PA['Barthel_resultados'] <= 20.0, SENSE_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA i total_SENSE_PA:", p_value_13)

P-valor entre total_AMB_PA i total_SENSE_PA: 0.14269651822055499


In [650]:
stat_23, p_value_23 = mannwhitneyu(AMB_PA_MECVV['Barthel_resultados'] <= 20.0, SENSE_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_AMB_PA_MECVV i total_SENSE_PA:", p_value_23)

P-valor entre total_AMB_PA_MECVV i total_SENSE_PA: 0.6663533091466043


## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [651]:
AMB_PA.loc[:, 'mna_resultats'] = pd.to_numeric(AMB_PA['mna_resultats'], errors='coerce')
AMB_PA_MECVV.loc[:, 'mna_resultats'] = pd.to_numeric(AMB_PA_MECVV['mna_resultats'], errors='coerce')
SENSE_PA.loc[:, 'mna_resultats'] = pd.to_numeric(SENSE_PA['mna_resultats'], errors='coerce')

### Mitjana

In [652]:
promedio_mna = AMB_PA['mna_resultats'].mean()
desviacion_estandar_mna = AMB_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [653]:
promedio_mna = AMB_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = AMB_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± nan


In [654]:
promedio_mna = SENSE_PA['mna_resultats'].mean()
desviacion_estandar_mna = SENSE_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultado)

Mitjana ± Desviació Estàndard del mna: 3.05 ± 5.92


#### Pvalor

##### Normalitat

In [655]:
stat, p_value = shapiro(AMB_PA['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [656]:
stat, p_value = shapiro(AMB_PA_MECVV['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [657]:
stat, p_value = shapiro(SENSE_PA['mna_resultats'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [658]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['mna_resultats'], AMB_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [659]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['mna_resultats'], SENSE_PA['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [660]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['mna_resultats'], SENSE_PA['mna_resultats'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors de malnutrició

##### AMB_PA

In [661]:
norm_AMB_PA = AMB_PA['mna_resultats'].between(24, 30)
Percentatge_norm_AMB_PA = norm_AMB_PA.mean() * 100
num_observaciones_norm_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de norm_AMB_PA: {Percentatge_norm_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_norm_AMB_PA}")

Percentatge de norm_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [662]:
malnutr_AMB_PA = AMB_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA = malnutr_AMB_PA.mean() * 100
num_observaciones_malnutr_AMB_PA = norm_AMB_PA.sum()

print(f"Percentatge de malnutr_AMB_PA: {Percentatge_malnutr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA: {num_observaciones_malnutr_AMB_PA}")

Percentatge de malnutr_AMB_PA: 0.00%
Nombre d'observacions en norm_AMB_PA: 0


In [663]:
malnut_AMB_PA = AMB_PA['mna_resultats'] < 17
Percentatge_malnut_AMB_PA = malnut_AMB_PA.mean() * 100
num_observaciones_malnut_AMB_PA = malnut_AMB_PA.sum()

print(f"Percentatge de malnut_AMB_PA: {Percentatge_malnut_AMB_PA:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA: {num_observaciones_malnut_AMB_PA}")

Percentatge de malnut_AMB_PA: 23.53%
Nombre d'observacions en malnut_AMB_PA: 4


##### AMB_PA_MECVV

In [664]:
norm_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'].between(24, 30)
Percentatge_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.mean() * 100
num_observaciones_norm_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de norm_AMB_PA_MECVV: {Percentatge_norm_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observaciones_norm_AMB_PA_MECVV}")

Percentatge de norm_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [665]:
malnutr_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_AMB_PA_MECVV = malnutr_AMB_PA_MECVV.mean() * 100
num_observaciones_malnutr_AMB_PA_MECVV = norm_AMB_PA_MECVV.sum()

print(f"Percentatge de malnutr_AMB_PA_MECVV: {Percentatge_malnutr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_AMB_PA_MECVV: {num_observaciones_malnutr_AMB_PA_MECVV}")

Percentatge de malnutr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en norm_AMB_PA_MECVV: 0


In [666]:
malnut_AMB_PA_MECVV = AMB_PA_MECVV['mna_resultats'] < 17
Percentatge_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.mean() * 100
num_observaciones_malnut_AMB_PA_MECVV = malnut_AMB_PA_MECVV.sum()

print(f"Percentatge de malnut_AMB_PA_MECVV: {Percentatge_malnut_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_AMB_PA_MECVV: {num_observaciones_malnut_AMB_PA_MECVV}")

Percentatge de malnut_AMB_PA_MECVV: 10.00%
Nombre d'observacions en malnut_AMB_PA_MECVV: 1


##### SENSE_PA

In [667]:
norm_SENSE_PA = SENSE_PA['mna_resultats'].between(24, 30)
Percentatge_norm_SENSE_PA = norm_SENSE_PA.mean() * 100
num_observaciones_norm_SENSE_PA = norm_SENSE_PA.sum()

print(f"Percentatge de norm_SENSE_PA: {Percentatge_norm_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en norm_SENSE_PA: {num_observaciones_norm_SENSE_PA}")

Percentatge de norm_SENSE_PA: 0.00%
Nombre d'observacions en norm_SENSE_PA: 0


In [668]:
malnutr_SENSE_PA = SENSE_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_SENSE_PA = malnutr_SENSE_PA.mean() * 100
num_observaciones_malnutr_SENSE_PA = malnutr_SENSE_PA.sum()

print(f"Percentatge de malnutr_SENSE_PA: {Percentatge_malnutr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SENSE_PA: {num_observaciones_malnutr_SENSE_PA}")

Percentatge de malnutr_SENSE_PA: 2.04%
Nombre d'observacions en malnutr_SENSE_PA: 1


In [669]:
malnut_SENSE_PA = SENSE_PA['mna_resultats'] < 17
Percentatge_malnut_SENSE_PA = malnut_SENSE_PA.mean() * 100
num_observaciones_malnut_SENSE_PA = malnut_SENSE_PA.sum()

print(f"Percentatge de malnut_SENSE_PA: {Percentatge_malnut_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SENSE_PA: {num_observaciones_malnut_SENSE_PA}")

Percentatge de malnut_SENSE_PA: 18.37%
Nombre d'observacions en malnut_SENSE_PA: 9


#### Pvalor

##### Normal

In [670]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'].between(24, 30), AMB_PA_MECVV['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre norm_AMB_PA i norm_AMB_PA_MECVV: 1.0


In [671]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'].between(24, 30), SENSE_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA i norm_SENSE_PA:", p_val_13)

P-valor entre norm_AMB_PA i norm_SENSE_PA: 1.0


In [672]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'].between(24, 30), SENSE_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA:", p_val_23)

P-valor entre norm_AMB_PA_MECVV i norm_SENSE_PA: 1.0


##### Risc de malnutrició

In [673]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'].between(17, 23.5), AMB_PA_MECVV['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV:", p_val_12)

P-valor entre malnutr_AMB_PA i malnutr_AMB_PA_MECVV: 1.0


In [674]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'].between(17, 23.5), SENSE_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA:", p_val_13)

P-valor entre malnutr_AMB_PA i malnutr_SENSE_PA: 0.5793276078637376


In [675]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'].between(17, 23.5), SENSE_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA:", p_val_23)

P-valor entre malnutr_AMB_PA_MECVV i malnutr_SENSE_PA: 0.6843175494727258


##### Malnutrició

In [676]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['mna_resultats'] <17 , 
                                AMB_PA_MECVV['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV:", p_val_12)

P-valor entre malnut_AMB_PA i norm_AMB_PA_MECVV: 0.4120280546315189


In [677]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['mna_resultats'] <17 , 
                                SENSE_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA i malnut_SENSE_PA:", p_val_13)

P-valor entre malnut_AMB_PA i malnut_SENSE_PA: 0.6549000779898075


In [678]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['mna_resultats'] <17 , 
                                SENSE_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA:", p_val_23)

P-valor entre malnut_AMB_PA_MECVV i malnut_SENSE_PA: 0.5341419377112933


## Index de l'EMINA

### Mitjana

In [679]:
promedio_emina = AMB_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = AMB_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 7.94 ± 3.21


In [680]:
promedio_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = AMB_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 7.60 ± 2.01


In [681]:
promedio_emina = SENSE_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = SENSE_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 5.64 ± 4.08


#### Pvalor

##### Normalitat

In [682]:
stat, p_value = shapiro(AMB_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9770
Valor p = 0.9248
El nombre de files segueix una distribució normal.


In [683]:
stat, p_value = shapiro(AMB_PA_MECVV['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8885
Valor p = 0.1630
El nombre de files segueix una distribució normal.


In [684]:
stat, p_value = shapiro(SENSE_PA['EMINA_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [685]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], AMB_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.7372065144531302


In [686]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [687]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'], SENSE_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors EMINA

##### AMB_PA

In [688]:
lowr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA = lowr_AMB_PA.mean() * 100
num_observaciones_lowr_AMB_PA = lowr_AMB_PA.sum()

print(f"Percentatge de lowr_AMB_PA: {Percentatge_lowr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA: {num_observaciones_lowr_AMB_PA}")

Percentatge de lowr_AMB_PA: 17.65%
Nombre d'observacions en lowr_AMB_PA: 3


In [689]:
modr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA = modr_AMB_PA.mean() * 100
num_observaciones_modr_AMB_PA = modr_AMB_PA.sum()

print(f"Percentatge de modr_AMB_PA: {Percentatge_modr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA: {num_observaciones_modr_AMB_PA}")

Percentatge de modr_AMB_PA: 64.71%
Nombre d'observacions en modr_AMB_PA: 11


In [690]:
highr_AMB_PA = AMB_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA = highr_AMB_PA.mean() * 100
num_observaciones_highr_AMB_PA = highr_AMB_PA.sum()

print(f"Percentatge de highr_AMB_PA: {Percentatge_highr_AMB_PA:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA: {num_observaciones_highr_AMB_PA}")

Percentatge de highr_AMB_PA: 17.65%
Nombre d'observacions en highr_AMB_PA: 3


##### AMB_PA_MECVV

In [691]:
lowr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.mean() * 100
num_observaciones_lowr_AMB_PA_MECVV = lowr_AMB_PA_MECVV.sum()

print(f"Percentatge de lowr_AMB_PA_MECVV: {Percentatge_lowr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_AMB_PA_MECVV: {num_observaciones_lowr_AMB_PA_MECVV}")

Percentatge de lowr_AMB_PA_MECVV: 20.00%
Nombre d'observacions en lowr_AMB_PA_MECVV: 2


In [692]:
modr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.mean() * 100
num_observaciones_modr_AMB_PA_MECVV = modr_AMB_PA_MECVV.sum()

print(f"Percentatge de modr_AMB_PA_MECVV: {Percentatge_modr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_AMB_PA_MECVV: {num_observaciones_modr_AMB_PA_MECVV}")

Percentatge de modr_AMB_PA_MECVV: 80.00%
Nombre d'observacions en modr_AMB_PA_MECVV: 8


In [693]:
highr_AMB_PA_MECVV = AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.mean() * 100
num_observaciones_highr_AMB_PA_MECVV = highr_AMB_PA_MECVV.sum()

print(f"Percentatge de highr_AMB_PA_MECVV: {Percentatge_highr_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_AMB_PA_MECVV: {num_observaciones_highr_AMB_PA_MECVV}")

Percentatge de highr_AMB_PA_MECVV: 0.00%
Nombre d'observacions en highr_AMB_PA_MECVV: 0


##### SENSE_PA

In [694]:
lowr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_SENSE_PA = lowr_SENSE_PA.mean() * 100
num_observaciones_lowr_SENSE_PA = lowr_SENSE_PA.sum()

print(f"Percentatge de lowr_SENSE_PA: {Percentatge_lowr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SENSE_PA: {num_observaciones_lowr_SENSE_PA}")

Percentatge de lowr_SENSE_PA: 44.90%
Nombre d'observacions en lowr_SENSE_PA: 22


In [695]:
modr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_SENSE_PA = modr_SENSE_PA.mean() * 100
num_observaciones_modr_SENSE_PA = modr_SENSE_PA.sum()

print(f"Percentatge de modr_SENSE_PA: {Percentatge_modr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modr_SENSE_PA: {num_observaciones_modr_SENSE_PA}")

Percentatge de modr_SENSE_PA: 40.82%
Nombre d'observacions en modr_SENSE_PA: 20


In [696]:
highr_SENSE_PA = SENSE_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_SENSE_PA = highr_SENSE_PA.mean() * 100
num_observaciones_highr_SENSE_PA = highr_SENSE_PA.sum()

print(f"Percentatge de highr_SENSE_PA: {Percentatge_highr_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en highr_SENSE_PA: {num_observaciones_highr_SENSE_PA}")

Percentatge de highr_SENSE_PA: 10.20%
Nombre d'observacions en highr_SENSE_PA: 5


#### Pvalor 

##### Risc baix

In [697]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.8874457485314164


In [698]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.028470032037651503


In [699]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SENSE_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.12129243251471264


##### Risc moderat

In [700]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.40237952012988454


In [701]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.09654829012396034


In [702]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SENSE_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.02066325652755892


##### Risc alt

In [703]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.08262239198015026


In [704]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.48486212369214565


In [705]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SENSE_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.02374418239053046


## Index de la canadenca 

### Mitjana

In [706]:
promedio_canad = AMB_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = AMB_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [707]:
promedio_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [708]:
promedio_canad = SENSE_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = SENSE_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 8.00 ± 1.41


#### Pvalor

##### Normalitat

In [709]:
stat, p_value = shapiro(AMB_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [710]:
stat, p_value = shapiro(AMB_PA_MECVV['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [711]:
stat, p_value = shapiro(SENSE_PA['Canadenca_sumatorios_comparados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El nombre de files segueix una distribució normal.


In [712]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], AMB_PA_MECVV['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [713]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [714]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'], SENSE_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Valors canadenca

##### AMB_PA 

In [715]:
mild_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA = mild_AMB_PA.mean() * 100
num_observaciones_mild_AMB_PA = mild_AMB_PA.sum()

print(f"Percentatge de mild_AMB_PA: {Percentatge_mild_AMB_PA:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA: {num_observaciones_mild_AMB_PA}")

Percentatge de mild_AMB_PA: 0.00%
Nombre d'observacions en mild_AMB_PA: 0


In [716]:
modneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA = modneu_AMB_PA.mean() * 100
num_observaciones_modneu_AMB_PA = modneu_AMB_PA.sum()

print(f"Percentatge de modneu_AMB_PA: {Percentatge_modneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA: {num_observaciones_modneu_AMB_PA}")

Percentatge de modneu_AMB_PA: 5.88%
Nombre d'observacions en modneu_AMB_PA: 1


In [717]:
sevneu_AMB_PA = AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA = sevneu_AMB_PA.mean() * 100
num_observaciones_sevneu_AMB_PA = sevneu_AMB_PA.sum()

print(f"Percentatge de sevneu_AMB_PA: {Percentatge_sevneu_AMB_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA: {num_observaciones_sevneu_AMB_PA}")

Percentatge de sevneu_AMB_PA: 0.00%
Nombre d'observacions en sevneu_AMB_PA: 0


##### AMB_PA_MECVV

In [718]:
mild_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.mean() * 100
num_observaciones_mild_AMB_PA_MECVV = mild_AMB_PA_MECVV.sum()

print(f"Percentatge de mild_AMB_PA_MECVV: {Percentatge_mild_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_AMB_PA_MECVV: {num_observaciones_mild_AMB_PA_MECVV}")

Percentatge de mild_AMB_PA_MECVV: 0.00%
Nombre d'observacions en mild_AMB_PA_MECVV: 0


In [719]:
modneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.mean() * 100
num_observaciones_modneu_AMB_PA_MECVV = modneu_AMB_PA_MECVV.sum()

print(f"Percentatge de modneu_AMB_PA_MECVV: {Percentatge_modneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_AMB_PA_MECVV: {num_observaciones_modneu_AMB_PA_MECVV}")

Percentatge de modneu_AMB_PA_MECVV: 10.00%
Nombre d'observacions en modneu_AMB_PA_MECVV: 1


In [720]:
sevneu_AMB_PA_MECVV = AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.mean() * 100
num_observaciones_sevneu_AMB_PA_MECVV = sevneu_AMB_PA_MECVV.sum()

print(f"Percentatge de sevneu_AMB_PA_MECVV: {Percentatge_sevneu_AMB_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_AMB_PA_MECVV: {num_observaciones_sevneu_AMB_PA_MECVV}")

Percentatge de sevneu_AMB_PA_MECVV: 0.00%
Nombre d'observacions en sevneu_AMB_PA_MECVV: 0


##### SENSE_PA

In [721]:
mild_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_SENSE_PA = mild_SENSE_PA.mean() * 100
num_observaciones_mild_SENSE_PA = mild_SENSE_PA.sum()

print(f"Percentatge de mild_SENSE_PA: {Percentatge_mild_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en mild_SENSE_PA: {num_observaciones_mild_SENSE_PA}")

Percentatge de mild_SENSE_PA: 0.00%
Nombre d'observacions en mild_SENSE_PA: 0


In [722]:
modneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_SENSE_PA = modneu_SENSE_PA.mean() * 100
num_observaciones_modneu_SENSE_PA = modneu_SENSE_PA.sum()

print(f"Percentatge de modneu_SENSE_PA: {Percentatge_modneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SENSE_PA: {num_observaciones_modneu_SENSE_PA}")

Percentatge de modneu_SENSE_PA: 4.08%
Nombre d'observacions en modneu_SENSE_PA: 2


In [723]:
sevneu_SENSE_PA = SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_SENSE_PA = sevneu_SENSE_PA.mean() * 100
num_observaciones_sevneu_SENSE_PA = sevneu_SENSE_PA.sum()

print(f"Percentatge de sevneu_SENSE_PA: {Percentatge_sevneu_SENSE_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SENSE_PA: {num_observaciones_sevneu_SENSE_PA}")

Percentatge de sevneu_SENSE_PA: 0.00%
Nombre d'observacions en sevneu_SENSE_PA: 0


#### Pvalor

##### Dèficit neurològic lleuger (mild)

In [724]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [725]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [726]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


##### Dèficit neurològic moderat

In [727]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.3700051180291794


In [728]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.6274490868971419


In [729]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SENSE_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.7870742763184819


##### Dèficit neurològic greu (sever)

In [730]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [731]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [732]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SENSE_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


## Infeccions respiratòries

##### AMB_PA 

In [733]:
mainresp_AMB_PA= (AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 41.18%


In [734]:
LRTI_AMB_PA= (AMB_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA:.2f}%")

Percentatge de LRTI: 17.65%


In [735]:
COPD_AMB_PA= (AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA:.2f}%")

Percentatge de exacerbacions de COPD: 29.41%


In [736]:
fibro_AMB_PA= (AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### AMB_PA_MECVV

In [737]:
mainresp_AMB_PA_MECVV= (AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_AMB_PA_MECVV:.2f}%")

Percentatge de infeccions respiratòries principals : 50.00%


In [738]:
LRTI_AMB_PA_MECVV= (AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_AMB_PA_MECVV:.2f}%")

Percentatge de LRTI: 20.00%


In [739]:
COPD_AMB_PA_MECVV = (AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_AMB_PA_MECVV:.2f}%")

Percentatge de exacerbacions de COPD: 30.00%


In [740]:
fibro_AMB_PA_MECVV = (AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_AMB_PA_MECVV:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


##### SENSE_PA

In [741]:
mainresp_SENSE_PA= (SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SENSE_PA:.2f}%")

Percentatge de infeccions respiratòries principals : 42.86%


In [742]:
LRTI_SENSE_PA= (SENSE_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SENSE_PA:.2f}%")

Percentatge de LRTI: 22.45%


In [743]:
COPD_SENSE_PA= (SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SENSE_PA:.2f}%")

Percentatge de exacerbacions de COPD: 32.65%


In [744]:
fibro_SENSE_PA= (SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SENSE_PA:.2f}%")

Percentatge de fibrosi pulmonar i fibrotòrax: 0.00%


#### Pvalor

##### Normalitat

In [745]:
stat, p_value = shapiro(AMB_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6317
Valor p = 0.00002144
El nombre de files no segueix una distribució normal.


In [746]:
stat, p_value = shapiro(AMB_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4702
Valor p = 0.00000073
El nombre de files no segueix una distribució normal.


In [747]:
stat, p_value = shapiro(AMB_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5785
Valor p = 0.00000650
El nombre de files no segueix una distribució normal.


In [748]:
stat, p_value = shapiro(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


C:\Users\jmiro\GitHub\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [749]:
stat, p_value = shapiro(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6553
Valor p = 0.00025396
El nombre de files no segueix una distribució normal.


In [750]:
stat, p_value = shapiro(AMB_PA_MECVV['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5093
Valor p = 0.00000467
El nombre de files no segueix una distribució normal.


In [751]:
stat, p_value = shapiro(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5942
Valor p = 0.00004713
El nombre de files no segueix una distribució normal.


In [752]:
stat, p_value = shapiro(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


In [753]:
stat, p_value = shapiro(SENSE_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6294
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [754]:
stat, p_value = shapiro(SENSE_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5158
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [755]:
stat, p_value = shapiro(SENSE_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5916
Valor p = 0.00000000
El nombre de files no segueix una distribució normal.


In [756]:
stat, p_value = shapiro(SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El nombre de files segueix una distribució normal.


##### Infeccions respiratòries principals 

In [757]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.6832089509594992


In [758]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.9113557832989896


In [759]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SENSE_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.6896369959432476


##### LRTI

In [760]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 0.9109318987037512


In [761]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0,alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.6863825106164784


In [762]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['LRTI_diagnosticada'] == 1.0, SENSE_PA['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.8770107151211659


##### Exacerbacions de COPD

In [763]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 1.0


In [764]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 0.8132303236327403


In [765]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SENSE_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 0.8809554446418177


##### Fibrosi pulmonar i fibrotòrax

In [766]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: 1.0


In [767]:
stat_13, p_val_13 = mannwhitneyu(AMB_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: 1.0


In [768]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SENSE_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: 1.0


## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [769]:
AMB_PA.loc[:, 'albumina'] = pd.to_numeric(AMB_PA['albumina'], errors='coerce')

promedio_alb = AMB_PA['albumina'].mean()
desviacion_estandar_alb = AMB_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.34 ± 0.38


In [770]:
AMB_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(AMB_PA_MECVV['albumina'], errors='coerce')

promedio_alb = AMB_PA_MECVV['albumina'].mean()
desviacion_estandar_alb = AMB_PA_MECVV['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.47 ± 0.55


In [771]:
SENSE_PA.loc[:, 'albumina'] = pd.to_numeric(SENSE_PA['albumina'], errors='coerce')

promedio_alb = SENSE_PA['albumina'].mean()
desviacion_estandar_alb = SENSE_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.58 ± 0.64


#### Pvalor

##### Normalitat

In [772]:
albpa = AMB_PA['albumina'].dropna()
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9311
Valor p = 0.56068160
El nombre de files segueix una distribució normal.


In [773]:
albAMBpamecvv = AMB_PA_MECVV['albumina'].dropna()
stat, p_value = shapiro(albAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9973
Valor p = 0.89985028
El nombre de files segueix una distribució normal.


In [774]:
albsensepa= SENSE_PA['albumina'].dropna()
stat, p_value = shapiro(albsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9778
Valor p = 0.92442182
El nombre de files segueix una distribució normal.


In [775]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['albumina'], AMB_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [776]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [777]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['albumina'], SENSE_PA['albumina'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Proteïnes totals

In [778]:
AMB_PA.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA['proteinas totales'], errors='coerce')

promedio_prot = AMB_PA['proteinas totales'].mean()
desviacion_estandar_prot = AMB_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.03 ± 0.73


In [779]:
AMB_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(AMB_PA_MECVV['proteinas totales'], errors='coerce')

promedio_prot = AMB_PA_MECVV['proteinas totales'].mean()
desviacion_estandar_prot = AMB_PA_MECVV['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.47 ± 0.76


In [780]:
SENSE_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SENSE_PA['proteinas totales'], errors='coerce')

promedio_prot = SENSE_PA['proteinas totales'].mean()
desviacion_estandar_prot = SENSE_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.12 ± 0.72


#### Pvalor

##### Normalitat

In [781]:
protAMBpa=AMB_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9780
Valor p = 0.95346065
El nombre de files segueix una distribució normal.


In [782]:
protAMBpamecvv=AMB_PA_MECVV['proteinas totales'].dropna()
stat, p_value = shapiro(protAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9975
Valor p = 0.99159177
El nombre de files segueix una distribució normal.


In [783]:
protsensepa=SENSE_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9753
Valor p = 0.86021925
El nombre de files segueix una distribució normal.


In [784]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['proteinas totales'], AMB_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [785]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [786]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['proteinas totales'], SENSE_PA['proteinas totales'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Hemoglobina

In [787]:
AMB_PA.loc[:, 'Hb'] = pd.to_numeric(AMB_PA['Hb'], errors='coerce')

promedio_hb = AMB_PA['Hb'].mean()
desviacion_estandar_hb = AMB_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 12.34 ± 1.77


In [788]:
AMB_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(AMB_PA_MECVV['Hb'], errors='coerce')

promedio_hb = AMB_PA_MECVV['Hb'].mean()
desviacion_estandar_hb = AMB_PA_MECVV['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.22 ± 1.40


In [789]:
SENSE_PA.loc[:, 'Hb'] = pd.to_numeric(SENSE_PA['Hb'], errors='coerce')

promedio_hb = SENSE_PA['Hb'].mean()
desviacion_estandar_hb = SENSE_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.22 ± 2.16


##### Pvalor

##### Normalitat

In [790]:
hbAMBpa=AMB_PA['Hb'].dropna()
stat, p_value = shapiro(hbAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9304
Valor p = 0.48501101
El nombre de files segueix una distribució normal.


In [791]:
hbAMBpamecvv=AMB_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8592
Valor p = 0.25743995
El nombre de files segueix una distribució normal.


In [792]:
hbsensepa=SENSE_PA['Hb'].dropna()
stat, p_value = shapiro(hbsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9551
Valor p = 0.37239632
El nombre de files segueix una distribució normal.


In [793]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['Hb'], AMB_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [794]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [795]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['Hb'], SENSE_PA['Hb'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Colesterol 

In [796]:
AMB_PA.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA['colesterol total'], errors='coerce')

promedio_col = AMB_PA['colesterol total'].mean()
desviacion_estandar_col = AMB_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 132.10 ± 34.83


In [797]:
AMB_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(AMB_PA_MECVV['colesterol total'], errors='coerce')

promedio_col = AMB_PA_MECVV['colesterol total'].mean()
desviacion_estandar_col = AMB_PA_MECVV['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 111.90 ± 28.37


In [798]:
SENSE_PA.loc[:, 'colesterol total'] = pd.to_numeric(SENSE_PA['colesterol total'], errors='coerce')

promedio_col = SENSE_PA['colesterol total'].mean()
desviacion_estandar_col = SENSE_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 145.27 ± 40.45


#### Pvalor

##### Normalitat 



In [799]:
colAMBpa=AMB_PA['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9887
Valor p = 0.99048226
El nombre de files segueix una distribució normal.


In [800]:
colAMBpamecvv=AMB_PA_MECVV['colesterol total'].dropna()
stat, p_value = shapiro(colAMBpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9462
Valor p = 0.55290474
El nombre de files segueix una distribució normal.


In [801]:
colsensepa=SENSE_PA['colesterol total'].dropna()
stat, p_value = shapiro(colsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9725
Valor p = 0.84373822
El nombre de files segueix una distribució normal.


In [802]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['colesterol total'], AMB_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [803]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [804]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['colesterol total'], SENSE_PA['colesterol total'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Leucòcits

In [805]:
AMB_PA.loc[:, 'leucos'] = pd.to_numeric(AMB_PA['leucos'], errors='coerce')

promedio_leu = AMB_PA['leucos'].mean()
desviacion_estandar_leu = AMB_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 12.69 ± 4.88


In [806]:
AMB_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(AMB_PA_MECVV['leucos'], errors='coerce')

promedio_leu = AMB_PA_MECVV['leucos'].mean()
desviacion_estandar_leu = AMB_PA_MECVV['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 13.08 ± 6.83


In [807]:
SENSE_PA.loc[:, 'leucos'] = pd.to_numeric(SENSE_PA['leucos'], errors='coerce')

promedio_leu = SENSE_PA['leucos'].mean()
desviacion_estandar_leu = SENSE_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard dels leucòcits: 11.52 ± 6.52


#### Pvalor 

##### Normalitat

In [808]:
leuambpa=AMB_PA['leucos'].dropna()
stat, p_value = shapiro(leuambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8482
Valor p = 0.09127190
El nombre de files segueix una distribució normal.


In [809]:
leuambpamecvv=AMB_PA_MECVV['leucos'].dropna()
stat, p_value = shapiro(leuambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8939
Valor p = 0.40148390
El nombre de files segueix una distribució normal.


In [810]:
leusensepa=SENSE_PA['leucos'].dropna()
stat, p_value = shapiro(leusensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8178
Valor p = 0.00096236
El nombre de files no segueix una distribució normal.


In [811]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['leucos'], AMB_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [812]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [813]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['leucos'], SENSE_PA['leucos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Limfòcits %

In [814]:
AMB_PA.loc[:, 'limfos'] = pd.to_numeric(AMB_PA['limfos'], errors='coerce')

promedio_limf = AMB_PA['limfos'].mean()
desviacion_estandar_limf = AMB_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 16.70 ± 12.29


In [815]:
AMB_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(AMB_PA_MECVV['limfos'], errors='coerce')

promedio_limf = AMB_PA_MECVV['limfos'].mean()
desviacion_estandar_limf = AMB_PA_MECVV['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 17.20 ± 9.35


In [816]:
SENSE_PA.loc[:, 'limfos'] = pd.to_numeric(SENSE_PA['limfos'], errors='coerce')

promedio_limf = SENSE_PA['limfos'].mean()
desviacion_estandar_limf = SENSE_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard dels limfòcits %: 19.50 ± 11.57


#### Pvalor

##### Normalitat

In [817]:
limfambpa=AMB_PA['limfos'].dropna()
stat, p_value = shapiro(limfambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8208
Valor p = 0.04757269
El nombre de files no segueix una distribució normal.


In [818]:
limfambpamecvv=AMB_PA_MECVV['limfos'].dropna()
stat, p_value = shapiro(limfambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9374
Valor p = 0.51710593
El nombre de files segueix una distribució normal.


In [819]:
limfsensepa=SENSE_PA['limfos'].dropna()
stat, p_value = shapiro(limfsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9290
Valor p = 0.13154504
El nombre de files segueix una distribució normal.


In [820]:
stat_12, p_val_12 = mannwhitneyu(AMB_PA['limfos'], AMB_PA_MECVV['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [821]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['limfos'], SENSE_PA['limfos'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [822]:
stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['limfos'], SENSE_PA['limfos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Proteïna C Reactiva

In [823]:
AMB_PA.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA['prot C react'], errors='coerce')

promedio_protcr = AMB_PA['prot C react'].mean()
desviacion_estandar_protcr = AMB_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 8.90 ± 5.47


In [824]:
AMB_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(AMB_PA_MECVV['prot C react'], errors='coerce')

promedio_protcr = AMB_PA_MECVV['prot C react'].mean()
desviacion_estandar_protcr = AMB_PA_MECVV['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 8.88 ± 3.63


In [825]:
SENSE_PA.loc[:, 'prot C react'] = pd.to_numeric(SENSE_PA['prot C react'], errors='coerce')

promedio_protcr = SENSE_PA['prot C react'].mean()
desviacion_estandar_protcr = SENSE_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteïna C reactiva: 6.16 ± 6.19


#### Pvalor

##### Normalitat

In [826]:
pcrambpa=AMB_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9423
Valor p = 0.60665876
El nombre de files segueix una distribució normal.


In [827]:
pcrambpamecvv=AMB_PA_MECVV['prot C react'].dropna()
stat, p_value = shapiro(pcrambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7886
Valor p = 0.08325234
El nombre de files segueix una distribució normal.


In [828]:
pcrsensepa=SENSE_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrsensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8698
Valor p = 0.00636559
El nombre de files no segueix una distribució normal.


In [829]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['prot C react'], AMB_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [830]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [831]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['prot C react'], SENSE_PA['prot C react'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Urea 

In [832]:
AMB_PA.loc[:, 'urea'] = pd.to_numeric(AMB_PA['urea'], errors='coerce')

promedio_urea = AMB_PA['urea'].mean()
desviacion_estandar_urea = AMB_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.17 ± 36.37


In [833]:
AMB_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(AMB_PA_MECVV['urea'], errors='coerce')

promedio_urea = AMB_PA_MECVV['urea'].mean()
desviacion_estandar_urea = AMB_PA_MECVV['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.37 ± 37.72


In [834]:
SENSE_PA.loc[:, 'urea'] = pd.to_numeric(SENSE_PA['urea'], errors='coerce')

promedio_urea = SENSE_PA['urea'].mean()
desviacion_estandar_urea = SENSE_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 55.03 ± 30.66


#### Pvalor

##### Normalitat

In [835]:
uambpa=AMB_PA['urea'].dropna()
stat, p_value = shapiro(uambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8842
Valor p = 0.17374314
El nombre de files segueix una distribució normal.


In [836]:
uambpamecvv=AMB_PA_MECVV['urea'].dropna()
stat, p_value = shapiro(uambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8991
Valor p = 0.42677381
El nombre de files segueix una distribució normal.


In [837]:
usensepa=SENSE_PA['urea'].dropna()
stat, p_value = shapiro(usensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8289
Valor p = 0.00115364
El nombre de files no segueix una distribució normal.


In [838]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['urea'], AMB_PA_MECVV['urea'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [839]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA:", p_val_13)

P-valor entre AMB_PA i SENSE_PA: nan


In [840]:
t_stat_23, p_val_23 = mannwhitneyu(AMB_PA_MECVV['urea'], SENSE_PA['urea'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Filtrat glomerular CDK-EPI

In [841]:
AMB_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = AMB_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = AMB_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 56.00 ± 12.65


In [842]:
AMB_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(AMB_PA_MECVV['FGE CDK-EPI'], errors='coerce')

promedio_fg = AMB_PA_MECVV['FGE CDK-EPI'].mean()
desviacion_estandar_fg = AMB_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 54.67 ± 13.32


In [843]:
SENSE_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SENSE_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = SENSE_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = SENSE_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI: 58.56 ± 18.40


#### Pvalor

##### Normalitat

In [844]:
fgepiambpa=AMB_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7690
Valor p = 0.01994941
El nombre de files no segueix una distribució normal.


In [845]:
fgepiambpamecvv=AMB_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiambpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8120
Valor p = 0.14355397
El nombre de files segueix una distribució normal.


In [846]:
fgepisensepa=SENSE_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisensepa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9160
Valor p = 0.14543070
El nombre de files segueix una distribució normal.


In [847]:
t_stat_12, p_val_12 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], AMB_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [848]:
t_stat_13, p_val_13 = mannwhitneyu(AMB_PA['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [849]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE CDK-EPI'], SENSE_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_val_23)

P-valor entre AMB_PA_MECVV i SENSE_PA: nan


### Filtrat glomerular MDRD

In [850]:
AMB_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = AMB_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = AMB_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 51.50 ± 0.71


In [851]:
AMB_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(AMB_PA_MECVV['FGE MDRD'], errors='coerce')

promedio_fgmd = AMB_PA_MECVV['FGE MDRD'].mean()
desviacion_estandar_fgmd = AMB_PA_MECVV['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: nan ± nan


In [852]:
SENSE_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SENSE_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = SENSE_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = SENSE_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtració glomerular MDRD: 37.67 ± 17.43


#### Pvalor

##### Normalitat

In [853]:
fgmd = AMB_PA['FGE MDRD'].dropna()
stat, p_value = shapiro(fgmd)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


ValueError: Data must be at least length 3.

In [854]:
stat, p_value = shapiro(AMB_PA_MECVV['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [855]:
stat, p_value = shapiro(SENSE_PA['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8324
Valor p = 0.11252528
El nombre de files segueix una distribució normal.


In [856]:
t_stat_12, p_val_12 = ttest_ind(AMB_PA['FGE MDRD'], AMB_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_12)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [857]:
t_stat_13, p_val_13 = ttest_ind(AMB_PA['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_val_13)

P-valor entre AMB_PA i AMB_PA_MECVV: nan


In [858]:
t_stat_23, p_val_23 = ttest_ind(AMB_PA_MECVV['FGE MDRD'], SENSE_PA['FGE MDRD'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i AMB_PA_MECVV:", p_val_23)

P-valor entre AMB_PA_MECVV i AMB_PA_MECVV: nan
